In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [36]:
from distgen.tools import stopwatch, vprint
from distgen.reader import reader
from distgen.writers import get_writer
from distgen.generator import generator
#from distgen.plot import plot_beam

import os

In [37]:
# Set up input data for test distgen module functions
inputfile = './test2.txt'
verbose = 2
output = 'gpt'
plots_on = False

os.path.exists(inputfile)

True

In [38]:
# Read input file using parser 
par = reader(inputfile,verbose,unit_registry)
par.read()
params = par.get_params()

Reading file './test2.txt'...done. Time Ellapsed: 569.11 microsecond
Parsing data...done. Time Ellapsed: 151.16 microsecond
particle_count:  ['100000']
dist_name:  ['gpt_particles']
output_format:  ['gpt']
total_charge:  ['1', '[pC]']
particle_type:  ['e-']
start_type:  ['cathode']
rand_type:  ['hammersley']
asci2gdf_binary:  ['asci2gdf']
xy_dist:  ['file', 'laser.prof.example.txt']
t_dist:  ['crystals']
laser_pulse_FWHM:  ['1.8', '[ps]']
crystal_length_1:  ['15.096', '[mm]']
crystal_angle_1:  ['0.6', '[deg]']
crystal_length_2:  ['7.548', '[mm]']
crystal_angle_2:  ['1.8', '[deg]']
crystal_length_3:  ['3.774', '[mm]']
crystal_angle_3:  ['-0.9', '[deg]']
crystal_length_4:  ['1.887', '[mm]']
crystal_angle_4:  ['-0.5', '[deg]']
avg_t:  ['0', '[ps]']
MTE:  ['140', '[meV]']


In [39]:
dict(params)  # display a typical input structure

{'particle_count': ['100000'],
 'dist_name': ['gpt_particles'],
 'output_format': ['gpt'],
 'total_charge': ['1', '[pC]'],
 'particle_type': ['e-'],
 'start_type': ['cathode'],
 'rand_type': ['hammersley'],
 'asci2gdf_binary': ['asci2gdf'],
 'xy_dist': ['file', 'laser.prof.example.txt'],
 't_dist': ['crystals'],
 'laser_pulse_FWHM': ['1.8', '[ps]'],
 'crystal_length_1': ['15.096', '[mm]'],
 'crystal_angle_1': ['0.6', '[deg]'],
 'crystal_length_2': ['7.548', '[mm]'],
 'crystal_angle_2': ['1.8', '[deg]'],
 'crystal_length_3': ['3.774', '[mm]'],
 'crystal_angle_3': ['-0.9', '[deg]'],
 'crystal_length_4': ['1.887', '[mm]'],
 'crystal_angle_4': ['-0.5', '[deg]'],
 'avg_t': ['0', '[ps]'],
 'MTE': ['140', '[meV]']}

In [40]:
# Make a beam distribution using the generator
gen = generator(verbose)
gen.parse_input(params)
beam,outfile = gen.get_beam()

Distribution format: gpt
Output file: gpt_particles

Creating beam distribution....
   Beam starting from: cathode.
   Total charge: 1.000 pC.
   Number of macroparticles: 100000.
   xy distribution: xy distribution file: 'laser.prof.example.txt
   t distribution: crystal temporal laser shaping
      crystal 1 length = 15.096 mm, angle = 0.600 deg
      crystal 2 length = 7.548 mm, angle = 1.800 deg
      crystal 3 length = 3.774 mm, angle = -0.900 deg
      crystal 4 length = 1.887 mm, angle = -0.500 deg
         applying crystal: 1
         applying crystal: 2
         applying crystal: 3
         applying crystal: 4
      Pulses propagated: min t = -23.905 ps, max t = 23.905 ps
   px distribution: Gaussian
      avg_px = 0.000 eV/c, sigma_px = 267.469 eV/c
   py distribution: Gaussian
      avg_py = 0.000 eV/c, sigma_py = 267.469 eV/c
   pz distribution: Gaussian
      avg_pz = 0.000 eV/c, sigma_pz = 267.469 eV/c
   Cathode start: fixing pz momenta to forward hemisphere
      avg_pz

In [41]:
# Print beam averages and standard deviations
beam.print_stats()

Beam stats:
   avg_x = -0.000 mm, sigma_x = 0.807 mm
   avg_y = 0.000 mm, sigma_y = 0.821 mm
   avg_z = 0.000 m, sigma_z = 0.000 m
   avg_px = -0.000 eV/c, sigma_px = 267.469 eV/c
   avg_py = 0.000 eV/c, sigma_py = 267.469 eV/c
   avg_pz = 213.421 eV/c, sigma_pz = 161.218 eV/c
   avg_t = -0.000 ps, sigma_t = 9.320 ps
   avg_q = -0.000 C, sigma_q = 0.000 C
   avg_nmacro = 62.415, sigma_nmacro = 0.000


In [42]:
# Print distribution to file using an output writer
file_writer = get_writer(output,outfile)
file_writer.write(beam,verbose,params)

watch.stop()
vprint("\nTotal time ellapsed: "+watch.print()+".",verbose>0,0,True)


Printing particles to 'gpt_particles'...
   Converting file to GDF: done. Time ellapsed: 8.45 millisecond.
...done. Time ellapsed: 810.05 millisecond.

Total time ellapsed: 1.12 kilosecond.


In [43]:
from distgen.pydistgen import run_distgen

In [44]:
# Use the combined run function to perform above tasks
beam = run_distgen(inputs=params,verbose=2)

**************************************************
                Dist Generator v 0.0
**************************************************
Distribution format: gpt
Output file: gpt_particles

Creating beam distribution....
   Beam starting from: cathode.
   Total charge: 1.000 pC.
   Number of macroparticles: 100000.
   xy distribution: xy distribution file: 'laser.prof.example.txt
   t distribution: crystal temporal laser shaping
      crystal 1 length = 15.096 mm, angle = 0.600 deg
      crystal 2 length = 7.548 mm, angle = 1.800 deg
      crystal 3 length = 3.774 mm, angle = -0.900 deg
      crystal 4 length = 1.887 mm, angle = -0.500 deg
         applying crystal: 1
         applying crystal: 2
         applying crystal: 3
         applying crystal: 4
      Pulses propagated: min t = -23.905 ps, max t = 23.905 ps
   px distribution: Gaussian
      avg_px = 0.000 eV/c, sigma_px = 267.469 eV/c
   py distribution: Gaussian
      avg_py = 0.000 eV/c, sigma_py = 267.469 eV/c
   pz distr